In [1]:
import numpy as np
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix
%matplotlib inline

Using TensorFlow backend.


In [3]:
vgg16_model=keras.applications.vgg16.VGG16(weights='D:/desk/harshak/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

In [5]:
model=Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [6]:
for layer in model.layers:
    layer.trainable=False

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [7]:
model.add(Dense(1000,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [5]:
'''
train_path='D:/Dataset/Images/set/train'
test_path='D:/Dataset/Images/set/test'
valid_path='D:/Dataset/Images/set/valid'
'''
train_path='D:/New folder/train'
test_path='D:/Kaggle/PetImages/test'
valid_path='D:/Kaggle/PetImages/valid'

In [7]:
model.compile(Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [6]:
train_batches=ImageDataGenerator().flow_from_directory(
    train_path,
    target_size=(224,224),
    class_mode='binary',
    batch_size=10
)
valid_batches=ImageDataGenerator().flow_from_directory(
    valid_path,
    target_size=(224,224),
    classes=['Dog','Cat'],
    batch_size=8
)
test_batches=ImageDataGenerator().flow_from_directory(
    test_path,
    target_size=(224,224),
    classes='binary',
    batch_size=10
)

Found 3797 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 0 images belonging to 6 classes.


In [18]:
model.fit_generator(train_batches,steps_per_epoch=4,validation_data=valid_batches,validation_steps=4,epochs=20,verbose=2)

Epoch 1/20
 - 2s - loss: 0.2882 - acc: 0.8163 - val_loss: 0.7749 - val_acc: 0.5625
Epoch 2/20
 - 1s - loss: 0.2753 - acc: 0.8500 - val_loss: 1.0131 - val_acc: 0.4375
Epoch 3/20
 - 1s - loss: 0.2107 - acc: 0.9475 - val_loss: 0.9190 - val_acc: 0.5312
Epoch 4/20
 - 1s - loss: 0.1727 - acc: 0.9212 - val_loss: 0.8415 - val_acc: 0.5000
Epoch 5/20
 - 1s - loss: 0.1826 - acc: 0.9475 - val_loss: 0.7671 - val_acc: 0.6250
Epoch 6/20
 - 1s - loss: 0.1175 - acc: 0.9475 - val_loss: 0.7291 - val_acc: 0.5938
Epoch 7/20
 - 1s - loss: 0.1272 - acc: 0.9750 - val_loss: 0.8411 - val_acc: 0.5000
Epoch 8/20
 - 1s - loss: 0.0784 - acc: 0.9737 - val_loss: 0.7812 - val_acc: 0.5625
Epoch 9/20
 - 1s - loss: 0.1554 - acc: 1.0000 - val_loss: 0.8276 - val_acc: 0.5312
Epoch 10/20
 - 1s - loss: 0.0608 - acc: 1.0000 - val_loss: 0.7530 - val_acc: 0.5938
Epoch 11/20
 - 1s - loss: 0.0879 - acc: 1.0000 - val_loss: 0.6651 - val_acc: 0.6250
Epoch 12/20
 - 1s - loss: 0.0727 - acc: 1.0000 - val_loss: 0.8263 - val_acc: 0.5000
E

In [8]:
model.load_weights('D:/test/weights-0.00002.h5')

ValueError: You are trying to load a weight file containing 55 layers into a model with 18 layers.

In [19]:
# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')


In [17]:
test_imgs,test_labels=next(test_batches)
plots(test_imgs,titles=test_labels)

ZeroDivisionError: integer division or modulo by zero

In [21]:
model.predict_generator(valid_batches,steps=1,verbose=1)

1/1 [==============================] - 1s 621ms/step


array([[1.2004394e-03, 9.9879956e-01],
       [8.8807827e-01, 1.1192178e-01],
       [3.6974248e-01, 6.3025755e-01],
       [9.4679487e-01, 5.3205136e-02],
       [9.6969730e-01, 3.0302657e-02],
       [8.1012951e-04, 9.9918991e-01],
       [3.2183257e-01, 6.7816740e-01],
       [6.2846345e-01, 3.7153661e-01]], dtype=float32)